<center><font size="+3"><strong>Air Quality in Nairobi🇿</strong></font></center>

In [10]:
import inspect
import time
import warnings
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import seaborn as sns
from pymongo import MongoClient
from sklearn.metrics import mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.ar_model import AutoReg

# Data Exploration

In [16]:
client = MongoClient(host="localhost", port=27017)
db = client["air-quality"]
dar = db["dar-es-salaam"]

Determination of the numbers assigned to all the sensor sites in the Dar es Salaam collection

In [17]:
sites = dar.distinct('metadata.site')
sites

[]

Determination which site in the Dar es Salaam collection has the most sensor readings

In [ ]:
result = dar.aggregate(
    [
        {"$group": {"_id": "$metadata.site", "count": {"$count": {}}}}
    ]
)
readings_per_site = list(result)
readings_per_site

## Importing data

Creating wrangle fuction that will extract the PM2.5 readings from the site that has the most total readings in the Dar es Salaam collection and then:
* Localizing reading time stamps to the timezone for `"Africa/Dar_es_Salaam"`
* Removing all outlier PM2.5 readings that are above 100. 
* Resampling the data to provide the mean PM2.5 reading for each hour.
* Imputing any missing values using the forward-will method. 
* Returning a Series `y`. 

In [ ]:
def wrangle(collection):
    results = collection.find(
        {"metadata.site": 11, "metadata.measurement": "P2"},
        projection={"P2": 1, "timestamp": 1, "_id": 0},
    )

    df = pd.DataFrame(results).set_index('timestamp')
    
    #Localize timzone
    df.index = df.index.tz_localize("UTC").tz_convert("Africa/Dar_es_Salaam")
    
    # Remove outliers
    df = df[df['P2'] < 100]
    
    # Resmaple to 1H window and fill NaN using forward fill method
    y = df['P2'].resample("1H").mean().fillna(method="ffill")
    
    
    return y

In [ ]:
y = wrangle(dar)
y.head()

Creating a time series plot of the readings in `y`

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
y.plot(xlabel='Date', ylabel='PM2.5 Level', title='Dar es Salaam PM2.5 Levels', ax=ax)

Creating and ploting rolling average of the readings in `y`

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
y.rolling(168).mean().plot(ax=ax, xlabel='Date', ylabel='PM2.5 Level', title='Dar es Salaam PM2.5 Levels, 7-Day Rolling Average')

Creating an ACF plot for the data in y

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
plot_acf(y, ax=ax)
plt.xlabel('Lag [hours]')
plt.ylabel('Correlation Coefficient')
plt.title('Dar es Salaam PM2.5 Readings, ACF')

Creating an PACF plot for the data in y

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
plot_pacf(y, ax=ax)
plt.xlabel('Lag [hours]')
plt.ylabel('Correlation Coefficient')
plt.title('Dar es Salaam PM2.5 Readings, PACF')

# Modeling

Split on train test data

In [ ]:
cutoff_test = int(len(y) * 0.90)
y_train = y.iloc[:cutoff_test]
y_test = y.iloc[cutoff_test:]

# Baseline

In [ ]:
y_train_mean = y_train.mean()
y_pred_baseline = [y_train_mean] * len(y_train)
mae_baseline = mean_absolute_error(y_train, y_pred_baseline)

print("Mean P2 Reading:", y_train_mean)
print("Baseline MAE:", mae_baseline)

## ARIMA model

Searching for `p` parameter and best model selection

In [ ]:
p_params = range(1, 31)
maes = []
i = 0
for p in p_params:
    start_time = time.time()
    model = AutoReg(y_train, lags=p).fit()
    elapsed_time = round(time.time() - start_time, 2)
    print(f"Trained AR model {p} in {elapsed_time} seconds.")
    y_pred = model.predict()
    mae = mean_absolute_error(y_train.iloc[p:], y_pred.iloc[p:])
    maes.append(mae)
mae_series = pd.Series(maes, name="mae", index=p_params)

mae_series.head()

In [ ]:
print("Best model: " mae_series.min()

In [ ]:
best_p = 25
best_model = AutoReg(y_train, lags=best_p).fit()

Plotting best model residuals

In [ ]:
y_train_resid = best_model.resid
y_train_resid.tail(10)

In [ ]:
y_train_resid.hist()
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.title("Best Model, Training Residuals");

ACF plot for residuals

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
plot_acf(y_train_resid, ax=ax)
plt.xlabel('Lag [hours]')
plt.ylabel('Correlation Coefficient')
plt.title('Dar es Salaam, Training Residuals ACF')

## Evaluation

Walk-forward validation for cerated model

In [ ]:
y_pred_wfv = pd.Series()
history = y_train.copy()
for i in range(len(y_test)):
    model = AutoReg(history, lags=28).fit()
    next_pred = model.forecast()
    y_pred_wfv = y_pred_wfv.append(next_pred)
    history = history.append(y_test[next_pred.index])

y_pred_wfv.head()

## Results

Poltting predicton results in comparision with test data

In [ ]:
df_pred_test = pd.DataFrame(
    {
        'Test data': y_test,
        'Prediction': y_pred_wfv
    }
)
fig = px.line(df_pred_test)
fig.update_layout(
    title="Dar es Salaam, WFV Predictions",
    xaxis_title="Date",
    yaxis_title="PM2.5 Level",
);
fig.show()